<a href="https://colab.research.google.com/github/GregoryLavor/analise-lotacao/blob/main/Analise_Lotacao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Análise de Dados utilizando PostgreSQL e Python (Pandas e PandaSQL)

####Configurando o setup

In [ ]:
!pip install pandasql

####Importando as bibliotecas

In [ ]:
import pandas as pd
import pandasql as ps

####Lendo as bases de dados

In [ ]:
tb_cargo = pd.read_csv ('https://raw.githubusercontent.com/GregoryLavor/analise_lotacao/main/Cargo.csv')
tb_departamento = pd.read_csv ('https://raw.githubusercontent.com/GregoryLavor/analise_lotacao/main/Departamento.csv')
tb_funcionario = pd.read_csv ('https://raw.githubusercontent.com/GregoryLavor/analise_lotacao/main/Funcionario.csv')
tb_lotacao = pd.read_csv ('https://raw.githubusercontent.com/GregoryLavor/analise_lotacao/main/Lotacao.csv')

#### Problema de negócio

#### 1. Listar nome (funcionário e cargo) por departamento com salário e pretensão salarial.

In [ ]:
query = """
select
cf.nome,
cc.nome as cargo,
cd.nome as departamento,
cf.pretensao_salarial,
lt.salario
from tb_lotacao as lt
inner join tb_funcionario as cf on cf.id = lt.id_funcionario
inner join tb_cargo as cc on cc.id = lt.id_cargo
inner join tb_departamento as cd on cd.id = lt.id_departamento
order by departamento,cargo asc
"""

ps.sqldf(query, locals())

,nome,cargo,departamento,pretensao_salarial,salario
0,Laura,Coordenador de Área,Administrativo,5400.0,6500.0
1,Juvencio,Coordenador de Área,Administrativo,6250.0,6500.0
2,Julia,Gestor Financeiro,Administrativo,9000.0,7000.0
3,Lucio,Gestor Financeiro,Administrativo,6000.0,7000.0
4,João,Programador,Administrativo,6000.0,7500.0
5,Antonio,Programador,Administrativo,6500.0,7500.0
6,Maria,Analista Contábil,Tecnologia da Informação,6500.0,5000.0
7,Lia,Analista Contábil,Tecnologia da Informação,5600.0,5000.0
8,Pedro,Analista de Dados,Tecnologia da Informação,5500.0,9000.0
9,Afonso,Analista de Dados,Tecnologia da Informação,5000.0,9000.0


#### 2. Listar os funcionários (nome e salário) que não sejam PCD e que possuam um salário maior que R$ 4.000,00 ordenados pelo maior salário.

In [ ]:
print('Tabela Lotação')
display(tb_lotacao.head(2))
print('Tabela Funcionário')
display(tb_funcionario.head(2))

Tabela Lotação


,id,id_funcionario,id_departamento,id_cargo,salario,cadastro,ativo
0,1,1,1,1,9000.0,2022-09-01,True
1,2,2,2,2,7500.0,2022-09-02,True


Tabela Funcionário


,id,nome,nascimento,pretensao_salarial,pcd
0,1,Pedro,1984-05-29,5500.0,True
1,2,João,1980-02-01,6000.0,False


In [ ]:
query = """
select
cf.nome,
case when cf.pcd = 0 then 'nao' else 'sim' end as pcd,
lt.salario
from tb_lotacao as lt
inner join tb_funcionario as cf on cf.id = lt.id_funcionario
where cf.pcd = False and lt.salario > 4000
order by salario desc
"""

ps.sqldf(query, locals())

,nome,pcd,salario
0,Morgana,nao,8000.0
1,Dorival,nao,8000.0
2,João,nao,7500.0
3,Antonio,nao,7500.0
4,Julia,nao,7000.0
5,Lucio,nao,7000.0
6,Laura,nao,6500.0
7,Maria,nao,5000.0
8,Lia,nao,5000.0


#### 3. Listar os cargos e suas respectivas médias salariais maiores que R$ 5.000,00.

In [ ]:
print('Tabela Cargo')
display(tb_cargo.head(2))
print('Tabela Lotação')
display(tb_lotacao.head(2))

Tabela Cargo


,id,nome
0,1,Analista de Dados
1,2,Programador


Tabela Lotação


,id,id_funcionario,id_departamento,id_cargo,salario,cadastro,ativo
0,1,1,1,1,9000.0,2022-09-01,True
1,2,2,2,2,7500.0,2022-09-02,True


In [ ]:
query = """
select
cc.nome as cargo,
avg(lt.salario) as media_salario
from tb_lotacao as lt
inner join tb_cargo as cc on cc.id = lt.id_cargo
group by
cc.nome
having media_salario > 5000
"""
ps.sqldf(query,locals())

,cargo,media_salario
0,Analista de Dados,9000.0
1,Analista de Infra,8000.0
2,Coordenador de Área,6500.0
3,Gestor Financeiro,7000.0
4,Programador,7500.0


#### 4. Listar os funcionários com salário e pretensão salarial, ordenando por quem recebe melhor em relação ao pretendido.

In [ ]:
print('Tabela Funcionário')
display(tb_funcionario.head(2))
print('Tabela Lotação')
display(tb_lotacao.head(2))

Tabela Funcionário


,id,nome,nascimento,pretensao_salarial,pcd
0,1,Pedro,1984-05-29,5500.0,True
1,2,João,1980-02-01,6000.0,False


Tabela Lotação


,id,id_funcionario,id_departamento,id_cargo,salario,cadastro,ativo
0,1,1,1,1,9000.0,2022-09-01,True
1,2,2,2,2,7500.0,2022-09-02,True


In [ ]:
query = """
select
cf.nome,
cc.nome as cargo,
cf.pretensao_salarial,
lt.salario,
lt.salario - cf.pretensao_salarial as diferenca,
case when salario > pretensao_salarial then 'Acima' else (case when salario = pretensao_salarial then 'Igual' else 'Abaixo' end) end as relacao
from tb_lotacao as lt
inner join tb_funcionario as cf on cf.id = lt.id_funcionario
inner join tb_cargo as cc on cc.id = lt.id_cargo
order by salario - pretensao_salarial desc
"""
ps.sqldf(query,locals())

,nome,cargo,pretensao_salarial,salario,diferenca,relacao
0,Dorival,Analista de Infra,3500.0,8000.0,4500.0,Acima
1,Afonso,Analista de Dados,5000.0,9000.0,4000.0,Acima
2,Pedro,Analista de Dados,5500.0,9000.0,3500.0,Acima
3,Morgana,Analista de Infra,5400.0,8000.0,2600.0,Acima
4,João,Programador,6000.0,7500.0,1500.0,Acima
5,Laura,Coordenador de Área,5400.0,6500.0,1100.0,Acima
6,Antonio,Programador,6500.0,7500.0,1000.0,Acima
7,Lucio,Gestor Financeiro,6000.0,7000.0,1000.0,Acima
8,Juvencio,Coordenador de Área,6250.0,6500.0,250.0,Acima
9,Arthur,Estagiário,1100.0,1100.0,0.0,Igual


#### 5. Listar quantos funcionários recebem acima, igual e abaixo do pretendido, com suas devidas porcentagens.

In [ ]:
query = """
select
sum(case when salario > pretensao_salarial then 1 else 0 end) as acima_pretendido,
round(cast(sum(case when salario > pretensao_salarial then 1 else 0 end) as real) * 100 / count(*) ,2) || '%' as perc_acima,
sum(case when salario = pretensao_salarial then 1 else 0 end) as igual_pretendido,
round(cast(sum(case when salario = pretensao_salarial then 1 else 0 end) as real) * 100 / count(*) ,2) || '%' as perc_igual,
sum(case when salario < pretensao_salarial then 1 else 0 end) as abaixo_pretendido,
round(cast(sum(case when salario < pretensao_salarial then 1 else 0 end) as real) * 100 / count(*) ,2) || '%' as perc_abaixo,
count(*) as total_funcionarios
from tb_lotacao as lt
inner join tb_funcionario as cf on cf.id = lt.id_funcionario
"""
ps.sqldf(query,locals())

,acima_pretendido,perc_acima,igual_pretendido,perc_igual,abaixo_pretendido,perc_abaixo,total_funcionarios
0,9,69.23%,1,7.69%,3,23.08%,13


#### 6. Listar a pretensão salarial média e a média de salario por cargo, ordenando pelo cargo que recebe melhor em relação ao pretendido.

In [ ]:
query = """
select
cc.nome as cargo,
avg(cf.pretensao_salarial) as media_pretensao,
avg(lt.salario) as media_salario,
avg(lt.salario) - avg(cf.pretensao_salarial) as diferenca
from tb_lotacao as lt
inner join tb_funcionario as cf on cf.id = lt.id_funcionario
inner join tb_cargo as cc on cc.id = lt.id_cargo
group by cargo
order by diferenca desc
"""
ps.sqldf(query,locals())

,cargo,media_pretensao,media_salario,diferenca
0,Analista de Dados,5250.0,9000.0,3750.0
1,Analista de Infra,4450.0,8000.0,3550.0
2,Programador,6250.0,7500.0,1250.0
3,Coordenador de Área,5825.0,6500.0,675.0
4,Estagiário,1100.0,1100.0,0.0
5,Gestor Financeiro,7500.0,7000.0,-500.0
6,Analista Contábil,6050.0,5000.0,-1050.0


#### 7. Listar a idade dos funcionários e classificá-los por faixa etária.

In [123]:
print('Tabela Funcionários')
display(tb_funcionario.head(2))

Tabela Funcionários


,id,nome,nascimento,pretensao_salarial,pcd
0,1,Pedro,1984-05-29,5500.0,True
1,2,João,1980-02-01,6000.0,False


In [125]:
query = """
select
cf.nome,
cf.nascimento,
date('now') as data_atual,
date('now') - date(cf.nascimento) - (strftime('%m-%d',date('now')) < strftime('%m-%d',date(cf.nascimento))) as idade
from tb_funcionario as cf
"""
ps.sqldf(query,locals())

,nome,nascimento,data_atual,idade
0,Pedro,1984-05-29,2024-04-22,39
1,João,1980-02-01,2024-04-22,44
2,Maria,2000-05-03,2024-04-22,23
3,Laura,2001-09-05,2024-04-22,22
4,Morgana,1990-04-07,2024-04-22,34
5,Julia,2003-08-23,2024-04-22,20
6,Afonso,2009-07-02,2024-04-22,14
7,Antonio,1970-11-01,2024-04-22,53
8,Lia,1969-08-30,2024-04-22,54
9,Juvencio,1968-07-02,2024-04-22,55


In [128]:
query = """
select
case when date('now') - date(cf.nascimento) - (strftime('%m-%d',date('now')) < strftime('%m-%d',date(cf.nascimento))) < 25 then '0 a 25 anos'
when date('now') - date(cf.nascimento) - (strftime('%m-%d',date('now')) < strftime('%m-%d',date(cf.nascimento))) between 26 and 35 then '26 a 35 anos'
when date('now') - date(cf.nascimento) - (strftime('%m-%d',date('now')) < strftime('%m-%d',date(cf.nascimento))) between 36 and 45 then '36 a 45 anos'
else 'mais de 45 anos' end as faixa_etaria,
count(*) as qtde_funcionarios
from tb_funcionario as cf
group by faixa_etaria
order by faixa_etaria asc
"""
ps.sqldf(query,locals())

,faixa_etaria,qtde_funcionarios
0,0 a 25 anos,7
1,26 a 35 anos,1
2,36 a 45 anos,2
3,mais de 45 anos,3


#### 8. Listar os funcionários com deficiência e listar quantos são, com a devida porcentagem em relação ao total de funcionários.

In [129]:
print('Tabela Funcionários')
display(tb_funcionario.head(2))

Tabela Funcionários


,id,nome,nascimento,pretensao_salarial,pcd
0,1,Pedro,1984-05-29,5500.0,True
1,2,João,1980-02-01,6000.0,False


In [133]:
query = """
select
cf.nome,
case when cf.pcd = true then 'sim' else 'nao' end as pcd
from tb_funcionario as cf
where pcd = true
"""
ps.sqldf(query,locals())

,nome,pcd
0,Pedro,sim
1,Afonso,sim
2,Juvencio,sim


In [141]:
query = """
select
sum(cf.pcd) as qtde_pcd,
round(cast(sum(cf.pcd) as real) * 100 / count(*) ,2) || '%' as perc_pcd,
count(*) as qtde_funcionarios
from tb_funcionario as cf
"""
ps.sqldf(query,locals())

,qtde_pcd,perc_pcd,qtde_funcionarios
0,3,23.08%,13
